# For colab (no web)

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import os
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class BookRecommender:
    def __init__(self):
        self.books = []  #list of dicts: {'title': ..., 'author': ..., 'text': ..., 'filepath': ...}
        self.vectorizer = TfidfVectorizer(stop_words='english', max_features=10000, ngram_range=(1, 2))
        self.tfidf_matrix = None
        self.similarity_matrix = None

    def load_books_from_folder(self, folder_path):
        if not os.path.exists(folder_path):
            raise FileNotFoundError(f"Folder '{folder_path}' not found.")

        pattern = re.compile(r"^(.*?)\s+by\s+(.*?)(?:\.txt)?$", re.IGNORECASE)
        txt_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.txt')]

        if not txt_files:
            print("No .txt files found in the folder.")
            return

        for filename in txt_files:
            filepath = os.path.join(folder_path, filename)
            match = pattern.match(os.path.splitext(filename)[0])

            if not match:
                print(f"Skipping file (incorrect name format): {filename}")
                continue

            title = match.group(1).strip()
            author = match.group(2).strip()

            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    text = f.read()
                words = text.split()
                text = ' '.join(words[:10000])  #use first 10k words (optional)

                self.books.append({
                    'title': title,
                    'author': author,
                    'text': text,
                    'filepath': filepath
                })
                print(f"Loaded: '{title}' by {author}")
            except Exception as e:
                print(f"Error reading {filename}: {e}")

        if not self.books:
            raise ValueError("No valid books loaded. Check file format and encoding.")

    def build_similarity_matrix(self):
        #fit TF-IDF and compute cosine similarity matrix
        if not self.books:
            raise ValueError("No books loaded. Call load_books_from_folder() first.")

        texts = [book['text'] for book in self.books]
        self.tfidf_matrix = self.vectorizer.fit_transform(texts)
        self.similarity_matrix = cosine_similarity(self.tfidf_matrix)

        print(f"Similarity matrix built for {len(self.books)} books.")

    def recommend_books(self, title_query, top_n=5):
        titles = [book['title'].lower() for book in self.books]
        query_lower = title_query.strip().lower()

        if query_lower not in titles:
            matches = [t for t in titles if query_lower in t]
            if matches:
                print(f"Did you mean: '{[self.books[titles.index(m)]['title'] for m in matches][0]}'?")
            raise ValueError(f"Book titled '{title_query}' not found. Available titles: {list(set(t['title'] for t in self.books))}")

        idx = titles.index(query_lower)
        sim_scores = self.similarity_matrix[idx]

        #sort by similarity descending, exclude self
        similar_indices = np.argsort(sim_scores)[::-1][1:top_n + 1]

        recommendations = []
        for i in similar_indices:
            recommendations.append({
                'title': self.books[i]['title'],
                'author': self.books[i]['author'],
                'similarity': float(sim_scores[i])
            })

        return recommendations

    def list_all_books(self):
        return [(book['title'], book['author']) for book in self.books]

In [33]:
if __name__ == "__main__":
    recommender = BookRecommender()

    BOOKS_FOLDER = "/content/drive/MyDrive/test"

    try:
        recommender.load_books_from_folder(BOOKS_FOLDER)

        print("\n📚 Books loaded:\n")
        for title, author in recommender.list_all_books():
            print(f"  • {title} by {author}")
        recommender.build_similarity_matrix()

        title = "The Scarlet Lette" # @param {type:"string"}
        print(f"\n🔎 Recommending books similar to '{test_title}':\n")
        recs = recommender.recommend_books(title, top_n=5)

        for i, rec in enumerate(recs, 1):
            print(f"{i}. {rec['title']} by {rec['author']} — Similarity: {rec['similarity']:.3f}")

    except Exception as e:
        print(f"Error: {e}")

Loaded: 'Hamlet' by William Shakespeare
Loaded: 'Hamlet' by William Shakespeare
Loaded: 'Moby Dick; Or, The Whale' by Herman Melville
Loaded: 'Moby Dick; Or, The Whale' by Herman Melville
Loaded: 'Romeo and Juliet' by William Shakespeare
Loaded: 'Romeo and Juliet' by William Shakespeare
Loaded: 'The Scarlet Letter' by Nathaniel Hawthorne
Loaded: 'The Scarlet Letter' by Nathaniel Hawthorne
Loaded: 'Alice's Adventures in Wonderland' by Lewis Carroll
Loaded: 'The Strange Case of Dr Jekyll and Mr Hyde' by Robert Louis Stevenson
Loaded: 'Alice's Adventures in Wonderland' by Lewis Carroll
Loaded: 'The Strange Case of Dr Jekyll and Mr Hyde' by Robert Louis StevensonLoaded: 'Frankenstein; Or, The Modern Prometheus' by Mary Wollstonecraft Shelley

Loaded: 'Frankenstein; Or, The Modern Prometheus' by Mary Wollstonecraft ShelleyLoaded: 'Crime and Punishment' by Fyodor Dostoyevsky

Loaded: 'Crime and Punishment' by Fyodor Dostoyevsky
Loaded: 'The Idiot' by Fyodor Dostoyevsky
Loaded: 'The Idiot' by

In [34]:
from google.colab import drive
drive.mount('/content/drive')

import os
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, request, render_template_string
import threading
import subprocess
import time

class BookRecommender:
    def __init__(self):
        self.books = []
        self.vectorizer = TfidfVectorizer(stop_words='english', max_features=10000, ngram_range=(1, 2))
        self.tfidf_matrix = None
        self.similarity_matrix = None

    def load_books_from_folder(self, folder_path):
        if not os.path.exists(folder_path):
            raise FileNotFoundError(f"Folder '{folder_path}' not found.")
        pattern = re.compile(r"^(.*?)\s+by\s+(.*?)(?:\.txt)?$", re.IGNORECASE)
        txt_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.txt')]
        if not txt_files:
            print("No .txt files found in the folder.")
            return
        for filename in txt_files:
            filepath = os.path.join(folder_path, filename)
            match = pattern.match(os.path.splitext(filename)[0])
            if not match:
                print(f"Skipping file (incorrect name format): {filename}")
                continue
            title = match.group(1).strip()
            author = match.group(2).strip()
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    text = f.read()
                words = text.split()
                text = ' '.join(words[:10000])
                self.books.append({
                    'title': title,
                    'author': author,
                    'text': text,
                    'filepath': filepath
                })
                print(f"Loaded: '{title}' by {author}")
            except Exception as e:
                print(f"Error reading {filename}: {e}")
        if not self.books:
            raise ValueError("No valid books loaded. Check file format and encoding.")

    def build_similarity_matrix(self):
        if not self.books:
            raise ValueError("No books loaded. Call load_books_from_folder() first.")
        texts = [book['text'] for book in self.books]
        self.tfidf_matrix = self.vectorizer.fit_transform(texts)
        self.similarity_matrix = cosine_similarity(self.tfidf_matrix)
        print(f"Similarity matrix built for {len(self.books)} books.")

    def recommend_books(self, title_query, top_n=5):
        titles = [book['title'].lower() for book in self.books]
        query_lower = title_query.strip().lower()
        if query_lower not in titles:
            matches = [t for t in titles if query_lower in t]
            if matches:
                idx = titles.index(matches[0])
                suggestion = f"{self.books[idx]['title']} by {self.books[idx]['author']}"
                raise ValueError(f"Did you mean: {suggestion}?")
            raise ValueError("Book not found.")
        idx = titles.index(query_lower)
        sim_scores = self.similarity_matrix[idx]
        similar_indices = np.argsort(sim_scores)[::-1][1:top_n + 1]
        recommendations = []
        for i in similar_indices:
            recommendations.append({
                'title': self.books[i]['title'],
                'author': self.books[i]['author'],
                'similarity': float(sim_scores[i])
            })
        return recommendations

    def list_all_books(self):
        return [(book['title'], book['author']) for book in self.books]

recommender = BookRecommender()
BOOKS_FOLDER = "/content/drive/MyDrive/test"

try:
    recommender.load_books_from_folder(BOOKS_FOLDER)
    recommender.build_similarity_matrix()
except Exception as e:
    print(f"Error loading books: {e}")

num_books = len(recommender.books)

app = Flask(__name__)

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>TF-IDF Book Recommender</title>
  <link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@400;700&display=swap" rel="stylesheet">
  <style>
    body {
      font-family: 'Montserrat', sans-serif;
      background: white;
      color: black;
      margin: 0;
      padding: 0;
      display: flex;
      justify-content: center;
      align-items: center;
      min-height: 100vh;
      flex-direction: column;
    }
    .container {
      text-align: center;
      width: 600px;
      max-width: 90vw;
    }
    h1 {
      font-size: 36px;
      margin-bottom: 20px;
      font-weight: 700;
    }
    .search-form {
      margin: 20px 0;
    }
    input[type="text"] {
      padding: 15px;
      width: 100%;
      max-width: 500px;
      font-size: 16px;
      border: 1px solid #ddd;
      border-radius: 24px;
      outline: none;
      box-shadow: 0 2px 6px rgba(0,0,0,0.1);
      transition: box-shadow 0.3s;
    }
    input[type="text"]:focus {
      box-shadow: 0 4px 10px rgba(0,0,0,0.15);
    }
    button {
      display: none; /* Hide submit button visually but keep form functional */
    }
    .info {
      font-size: 16px;
      margin-top: 10px;
      color: #444;
    }
    .error {
      color: #d32f2f;
      margin-top: 10px;
      font-style: italic;
    }
    .results {
      margin-top: 30px;
      width: 100%;
      border-collapse: collapse;
    }
    .results td, .results th {
      padding: 12px 10px;
      text-align: left;
      border: none;
    }
    .results tr:nth-child(even) {
      background-color: transparent;
    }
    .results th {
      font-weight: 700;
      color: #000;
    }
  </style>
</head>
<body>
  <div class="container">
    <h1>TF-IDF Book Recommender</h1>

    <form method="post" class="search-form">
      <input type="text" name="title" placeholder="Search for a book..." value="{{ request.form.title if request.method == 'POST' }}" autocomplete="off">
    </form>

    {% if results %}
      <p class="info">Number of books in database: {{ num_books }}</p>
      <table class="results">
        <thead>
          <tr>
            <th>#</th>
            <th>Title</th>
            <th>Author</th>
            <th>Similarity</th>
          </tr>
        </thead>
        <tbody>
          {% for rec in results %}
            <tr>
              <td>{{ loop.index }}</td>
              <td>{{ rec.title }}</td>
              <td>{{ rec.author }}</td>
              <td>{{ "%.3f" % rec.similarity }}</td>
            </tr>
          {% endfor %}
        </tbody>
      </table>
    {% elif error %}
      <p class="error">{{ error }}</p>
    {% else %}
      <p class="info">Number of books in database: {{ num_books }}</p>
    {% endif %}
  </div>
</body>
</html>
'''

@app.route("/", methods=["GET", "POST"])
def index():
    results = None
    error = None
    if request.method == "POST":
        title_input = request.form["title"].strip()
        if title_input:
            try:
                results = recommender.recommend_books(title_input, top_n=5)
            except Exception as e:
                error = str(e)
        else:
            error = "Please enter a book title."
    return render_template_string(HTML_TEMPLATE,
                                num_books=num_books,
                                results=results,
                                error=error,
                                request=request)

def run_app():
    app.run(host="0.0.0.0", port=5000)

threading.Thread(target=run_app, daemon=True).start()

print("Installing localtunnel...")
os.system("npm install -g localtunnel")

print("Starting localtunnel...")
tunnel_proc = subprocess.Popen(["lt", "--port", "5000"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
time.sleep(3)

public_url = None
try:
    public_url = tunnel_proc.stdout.readline().strip()
    if not public_url:
        public_url = tunnel_proc.stderr.readline().strip()
except:
    pass

if public_url and "https://" in public_url:
    print(f"\n🌟 Public URL: {public_url}")
else:
    lines = []
    for _ in range(10):
        line = tunnel_proc.stdout.readline().strip()
        if line:
            lines.append(line)
    for line in lines:
        if "https://" in line:
            print(f"\n🌐 Public URL: {line}")
            print(f"\n🌐 Password:")
            break
    else:
        print("\n❌ Could not retrieve public URL.")

Loaded: 'The Idiot' by Fyodor Dostoyevsky
Loaded: 'War and Peace' by Leo Tolstoy
Loaded: 'Anna Karenina' by Leo Tolstoy
Loaded: 'Treasure Island' by Robert Louis Stevenson
Loaded: 'Dead Souls' by Nikolai Gogol
Loaded: 'Oblomov' by Ivan Goncharov
Loaded: 'Hamlet' by William Shakespeare
Loaded: 'The Seven Who Were Hanged' by Leonid Andreyev
Loaded: 'Moby Dick; Or, The Whale' by Herman Melville
Loaded: 'Romeo and Juliet' by William Shakespeare
Loaded: 'The Scarlet Letter' by Nathaniel Hawthorne
Loaded: 'Alice's Adventures in Wonderland' by Lewis Carroll
Loaded: 'The Strange Case of Dr Jekyll and Mr Hyde' by Robert Louis Stevenson
Loaded: 'Frankenstein; Or, The Modern Prometheus' by Mary Wollstonecraft Shelley
Loaded: 'Crime and Punishment' by Fyodor Dostoyevsky
Loaded: 'The Idiot' by Fyodor Dostoyevsky
Loaded: 'War and Peace' by Leo Tolstoy
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded: 'Anna Karenin

Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Loaded: 'War and Peace' by Leo Tolstoy
Loaded: 'Hamlet' by William Shakespeare
Loaded: 'Moby Dick; Or, The Whale' by Herman MelvilleLoaded: 'Anna Karenina' by Leo Tolstoy

Loaded: 'Romeo and Juliet' by William Shakespeare
Loaded: 'Treasure Island' by Robert Louis Stevenson
Loaded: 'The Scarlet Letter' by Nathaniel Hawthorne
Loaded: 'Dead Souls' by Nikolai Gogol
Loaded: 'Alice's Adventures in Wonderland' by Lewis Carroll
Loaded: 'Oblomov' by Ivan Goncharov
Loaded: 'The Strange Case of Dr Jekyll and Mr Hyde' by Robert Louis Stevenson
Loaded: 'The Seven Who Were Hanged' by Leonid Andreyev
Loaded: 'Frankenstein; Or, The Modern Prometheus' by Mary Wollstonecraft Shelley
Loaded: 'Crime and Punishment' by Fyodor Dostoyevsky
Loaded: 'Hamlet' by William Shakespeare
Loaded: 'The Idiot' by Fyodor Dostoyevsky
Loaded: 'Moby Dick; Or, The Whale' by Herman Melville
Loaded: 'Romeo and Juliet' by William Shakespeare
Loaded: 'War and Peace' by Leo Tolstoy
Loaded: 'The Scarlet Letter' by Nathaniel Hawtho

In [35]:
!curl https://loca.lt/mytunnelpassword

Loaded: 'Hamlet' by William Shakespeare
Loaded: 'Moby Dick; Or, The Whale' by Herman Melville
Loaded: 'Romeo and Juliet' by William Shakespeare
Loaded: 'The Scarlet Letter' by Nathaniel Hawthorne
Loaded: 'Alice's Adventures in Wonderland' by Lewis Carroll
Loaded: 'The Strange Case of Dr Jekyll and Mr Hyde' by Robert Louis Stevenson
Loaded: 'Frankenstein; Or, The Modern Prometheus' by Mary Wollstonecraft Shelley
Loaded: 'Crime and Punishment' by Fyodor Dostoyevsky
34.74.120.135

# Dynamic + QR

In [38]:
with open('/content/drive/MyDrive/tf-idf/stop-words-russian.txt', 'r') as file:
    russian_stop_words = [line.strip() for line in file]

print(russian_stop_words)

['\ufeffа', 'е', 'и', 'ж', 'м', 'о', 'на', 'не', 'ни', 'об', 'но', 'он', 'мне', 'мои', 'мож', 'она', 'они', 'оно', 'мной', 'много', 'многочисленное', 'многочисленная', 'многочисленные', 'многочисленный', 'мною', 'мой', 'мог', 'могут', 'можно', 'может', 'можхо', 'мор', 'моя', 'моё', 'мочь', 'над', 'нее', 'оба', 'нам', 'нем', 'нами', 'ними', 'мимо', 'немного', 'одной', 'одного', 'менее', 'однажды', 'однако', 'меня', 'нему', 'меньше', 'ней', 'наверху', 'него', 'ниже', 'мало', 'надо', 'один', 'одиннадцать', 'одиннадцатый', 'назад', 'наиболее', 'недавно', 'миллионов', 'недалеко', 'между', 'низко', 'меля', 'нельзя', 'нибудь', 'непрерывно', 'наконец', 'никогда', 'никуда', 'нас', 'наш', 'нет', 'нею', 'неё', 'них', 'мира', 'наша', 'наше', 'наши', 'ничего', 'начала', 'нередко', 'несколько', 'обычно', 'опять', 'около', 'мы', 'ну', 'нх', 'от', 'отовсюду', 'особенно', 'нужно', 'очень', 'отсюда', 'в', 'во', 'вон', 'вниз', 'внизу', 'вокруг', 'вот', 'восемнадцать', 'восемнадцатый', 'восемь', 'восьмой'

In [39]:
from google.colab import drive
drive.mount('/content/drive')

import os
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, request, render_template_string
import threading
import subprocess
import time
import socket
from urllib.parse import quote

class BookRecommender:
    def __init__(self):
        self.books = []
        self.loaded_filenames = set()
        self.vectorizer = TfidfVectorizer(stop_words=russian_stop_words, max_features=10000, ngram_range=(1, 2))
        self.tfidf_matrix = None
        self.similarity_matrix = None
        self.lock = threading.Lock()

    def load_books_from_folder(self, folder_path):
        if not os.path.exists(folder_path):
            raise FileNotFoundError(f"Folder '{folder_path}' not found.")

        pattern = re.compile(r"^(.*?)\s+by\s+(.*?)(?:\.txt)?$", re.IGNORECASE)
        current_txt_files = {f for f in os.listdir(folder_path) if f.lower().endswith('.txt')}
        previous_filenames = set(self.loaded_filenames)

        # Detect removed files
        removed_files = previous_filenames - current_txt_files
        new_books_found = False
        books_removed = False

        # Remove books from collection if their file was deleted
        if removed_files:
            with self.lock:
                # Filter out books that came from removed files
                self.books = [book for book in self.books if os.path.basename(book['filepath']) not in removed_files]
                # Update loaded filenames
                self.loaded_filenames = {f for f in self.loaded_filenames if f not in removed_files}
                print(f"🗑️ Removed {len(removed_files)} books (files deleted): {removed_files}")
                books_removed = True

        # Now process new or updated files
        for filename in current_txt_files:
            if filename in self.loaded_filenames:
                continue  # Already loaded

            filepath = os.path.join(folder_path, filename)
            match = pattern.match(os.path.splitext(filename)[0])
            if not match:
                print(f"Skipping file (incorrect name format): {filename}")
                self.loaded_filenames.add(filename)
                continue

            title = match.group(1).strip()
            author = match.group(2).strip()
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    text = f.read()
                words = text.split()
                text = ' '.join(words[:10000])  # Limit size
                book_data = {
                    'title': title,
                    'author': author,
                    'text': text,
                    'filepath': filepath
                }
                with self.lock:
                    self.books.append(book_data)
                    self.loaded_filenames.add(filename)
                print(f"✅ Loaded new book: '{title}' by {author}")
                new_books_found = True
            except Exception as e:
                print(f"Error reading {filename}: {e}")
                self.loaded_filenames.add(filename)

        # Rebuild similarity matrix only if something changed
        if new_books_found or books_removed:
            print("🔄 Rebuilding similarity matrix due to added/removed books...")
            self.build_similarity_matrix()

    def build_similarity_matrix(self):
        with self.lock:
            if not self.books:
                return
            texts = [book['text'] for book in self.books]
            self.tfidf_matrix = self.vectorizer.fit_transform(texts)
            self.similarity_matrix = cosine_similarity(self.tfidf_matrix)
            print(f"📊 Similarity matrix updated for {len(self.books)} books.")

    def recommend_books(self, title_query, top_n=5):
        with self.lock:
            titles = [book['title'].lower() for book in self.books]
            query_lower = title_query.strip().lower()
            if query_lower not in titles:
                matches = [t for t in titles if query_lower in t]
                if matches:
                    idx = titles.index(matches[0])
                    suggestion = f"{self.books[idx]['title']} by {self.books[idx]['author']}"
                    raise ValueError(f"Did you mean: {suggestion}?")
                raise ValueError("Book not found.")
            idx = titles.index(query_lower)
            sim_scores = self.similarity_matrix[idx]
            similar_indices = np.argsort(sim_scores)[::-1][1:top_n + 1]
            recommendations = []
            for i in similar_indices:
                recommendations.append({
                    'title': self.books[i]['title'],
                    'author': self.books[i]['author'],
                    'similarity': float(sim_scores[i])
                })
            return recommendations

    def list_all_books(self):
        return [(book['title'], book['author']) for book in self.books]

recommender = BookRecommender()
BOOKS_FOLDER = "/content/drive/MyDrive/tf-idf/txt"

try:
    print("🔍 Loading initial books...")
    recommender.load_books_from_folder(BOOKS_FOLDER)
except Exception as e:
    print(f"Error loading initial books: {e}")

num_books = len(recommender.books)

def poll_for_new_books():
    while True:
        time.sleep(5)
        try:
            recommender.load_books_from_folder(BOOKS_FOLDER)
        except Exception as e:
            print(f"Error during polling: {e}")

threading.Thread(target=poll_for_new_books, daemon=True).start()


app = Flask(__name__)

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>TF-IDF Book Recommender</title>
  <link href="https://fonts.googleapis.com/css2?family=Montserrat:wght@400;700&display=swap" rel="stylesheet">
  <style>
    body {
      font-family: 'Montserrat', sans-serif;
      background: white;
      color: black;
      margin: 0;
      padding: 0;
      display: flex;
      justify-content: center;
      align-items: center;
      min-height: 100vh;
      flex-direction: column;
    }
    .container {
      text-align: center;
      width: 600px;
      max-width: 90vw;
      padding: 20px;
    }
    h1 {
      font-size: 36px;
      margin-bottom: 20px;
      font-weight: 700;
    }
    .search-form {
      margin: 20px 0;
    }
    input[type="text"] {
      padding: 15px;
      width: 100%;
      max-width: 500px;
      font-size: 16px;
      border: 1px solid #ddd;
      border-radius: 24px;
      outline: none;
      box-shadow: 0 2px 6px rgba(0,0,0,0.1);
      transition: box-shadow 0.3s;
    }
    input[type="text"]:focus {
      box-shadow: 0 4px 10px rgba(0,0,0,0.15);
    }
    button {
      display: none;
    }
    .info {
      font-size: 16px;
      margin-top: 10px;
      color: #444;
    }
    .error {
      color: #d32f2f;
      margin-top: 10px;
      font-style: italic;
    }
    .results {
      margin-top: 30px;
      width: 100%;
      border-collapse: collapse;
    }
    .results td, .results th {
      padding: 12px 10px;
      text-align: left;
      border: none;
    }
    .results tr:nth-child(even) {
      background-color: rgba(0, 0, 0, 0.03);
    }
    .results th {
      font-weight: 700;
      color: #000;
      background-color: transparent;
    }
    .results tbody tr:hover {
      background-color: rgba(0, 0, 0, 0.05);
    }
  </style>
</head>
<body>

  <!-- Main Content -->
  <div class="container">
    <h1>TF-IDF Book Recommender</h1>
    <form method="post" class="search-form">
      <input
        type="text"
        name="title"
        placeholder="Search for a book..."
        value="{{ request.form.title if request.method == 'POST' }}"
        autocomplete="off"
        aria-label="Search for a book"
      >
    </form>
    <p class="info">Number of books in database: <strong><span id="bookCount">{{ num_books }}</span></strong></p>

    {% if results %}
      <table class="results">
        <thead>
          <tr>
            <th>#</th>
            <th>Title</th>
            <th>Author</th>
            <th>Similarity</th>
          </tr>
        </thead>
        <tbody>
          {% for rec in results %}
            <tr>
              <td>{{ loop.index }}</td>
              <td>{{ rec.title }}</td>
              <td>{{ rec.author }}</td>
              <td>{{ "%.3f" % rec.similarity }}</td>
            </tr>
          {% endfor %}
        </tbody>
      </table>
    {% elif error %}
      <p class="error">{{ error }}</p>
    {% else %}
      <p class="info">Enter a book title to get recommendations.</p>
    {% endif %}
  </div>

  <!-- QR Code Overlay (Bottom Right Corner) -->
  <div id="qr-overlay" style="
    position: fixed;
    bottom: 20px;
    right: 20px;
    background: white;
    border: 1px solid #ddd;
    border-radius: 12px;
    padding: 12px;
    box-shadow: 0 4px 16px rgba(0,0,0,0.15);
    width: 180px;
    text-align: center;
    font-family: 'Montserrat', sans-serif;
    z-index: 1000;
    font-size: 12px;
  ">
    <div id="qrcode" style="display: inline-block;"></div>
    <p style="margin: 8px 0 0; font-size: 18px; color: #444;">
      password: {{ password_ip }}
    </p>
  </div>

  <!-- Load QR Code Generator (Client-Side) -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/qrcodejs/1.0.0/qrcode.min.js"></script>
  <script>
    // Generate QR code for public URL
    new QRCode(document.getElementById("qrcode"), {
      text: "{{ public_url }}",
      width: 160,
      height: 160,
      colorDark: "#000",
      colorLight: "#fff"
    });
  </script>

  <!-- Auto-update Book Count -->
  <script>
    function updateBookCount() {
      fetch('/api/book_count')
        .then(response => {
          if (!response.ok) throw new Error("Network error");
          return response.json();
        })
        .then(data => {
          const countSpan = document.getElementById('bookCount');
          countSpan.textContent = data.count;
        })
        .catch(err => {
          // Silently fail
        });
    }

    updateBookCount();
    setInterval(updateBookCount, 2000);
  </script>

</body>
</html>
'''

@app.route("/", methods=["GET", "POST"])
def index():
    with recommender.lock:
        current_num_books = len(recommender.books)
    results = None
    error = None
    if request.method == "POST":
        title_input = request.form["title"].strip()
        if title_input:
            try:
                results = recommender.recommend_books(title_input, top_n=5)
            except Exception as e:
                error = str(e)
        else:
            error = "Please enter a book title."

    return render_template_string(
        HTML_TEMPLATE,
        num_books=current_num_books,
        results=results,
        error=error,
        request=request,
        public_url=public_url,
        password_ip=password_ip
    )

@app.route("/api/book_count")
def book_count():
    with recommender.lock:
        count = len(recommender.books)
    return {"count": count}

def run_app():
    app.run(host="0.0.0.0", port=5000)

threading.Thread(target=run_app, daemon=True).start()


print("📦 Installing localtunnel...")
os.system("npm install -g localtunnel")

print("🔁 Starting localtunnel...")
tunnel_proc = subprocess.Popen(["lt", "--port", "5000"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
time.sleep(3)

public_url = None
try:
    output_line = tunnel_proc.stdout.readline().strip()
    if not output_line or "https://" not in output_line:
        output_line = tunnel_proc.stderr.readline().strip()
    if "https://" in output_line:
        public_url = output_line
except:
    pass

if not public_url:
    for _ in range(10):
        line = tunnel_proc.stdout.readline().strip()
        if "https://" in line:
            public_url = line
            break

if public_url:
    public_url = public_url.split(" ")[-1].strip()  # Extract clean URL
    print(f"\n🌟 Public URL: {public_url}")
else:
    print("\n❌ Could not retrieve public URL.")
    public_url = "https://example.com"

import requests

def get_external_ip():
    try:
        return requests.get("https://api.ipify.org", timeout=5).text.strip()
    except:
        return "unable-to-get-ip"

password_ip = get_external_ip()
print(f"🔐 Access Password (IP): {password_ip}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔍 Loading initial books...
✅ Loaded new book: 'Сахарный Кремль' by Владимир Сорокин
✅ Loaded new book: 'Убийство на улице Морг' by Эдгар Аллан По
✅ Loaded new book: 'Норма' by Владимир Сорокин
✅ Loaded new book: 'Превращение' by Франц Кафка
✅ Loaded new book: 'Бедная Лиза' by Николай Карамзин
✅ Loaded new book: 'Москва-Петушки' by Венедикт Ерофеев
✅ Loaded new book: 'Евгений Онегин' by Александр Пушкин
✅ Loaded new book: 'Обломов' by Иван Гончаров
✅ Loaded new book: 'Тарас Бульба' by Николай Гоголь
✅ Loaded new book: 'Фауст' by Иоганн Вольфганг Гёте
✅ Loaded new book: 'Тень над Иннсмутом' by Говард Лавкрафт
✅ Loaded new book: 'Анна Каренина' by Лев Толстой
✅ Loaded new book: 'Голубое сало' by Владимир Сорокин
✅ Loaded new book: 'Господин из Сан-Франциско' by Иван Бунин
✅ Loaded new book: 'Метель' by Владимир Сорокин
✅ Loaded new book: 'Страдания юного 

Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


🔁 Starting localtunnel...

🌟 Public URL: https://chilly-crabs-trade.loca.lt
🔐 Access Password (IP): 34.74.120.135


# Book autodownload + cleanup

In [ ]:
from google.colab import drive
import os
import requests
import zipfile
import shutil
import re
import time
import chardet

def read_with_encoding(filepath):
    with open(filepath, 'rb') as f:
        raw = f.read()
        result = chardet.detect(raw)
        encoding = result['encoding']
    with open(filepath, 'r', encoding=encoding, errors='ignore') as f:
        return f.read()

drive.mount('/content/drive')

base_dir = '/content/drive/MyDrive/tf-idf'
zip_dir = os.path.join(base_dir, 'zip')
txt_dir = os.path.join(base_dir, 'txt')

os.makedirs(zip_dir, exist_ok=True)
os.makedirs(txt_dir, exist_ok=True)

books_by_author = {
    "Александр Пушкин": ["Капитанская дочка", "Дубровский", "Евгений Онегин"],
    "Николай Гоголь": ["Мёртвые души", "Вий", "Тарас Бульба"],
    "Михаил Лермонтов": ["Герой нашего времени", "Мцыри"],
    "Иван Тургенев": ["Отцы и дети", "Муму"],
    "Фёдор Достоевский": ["Преступление и наказание", "Идиот", "Братья Карамазовы", "Бесы"],
    "Лев Толстой": ["Война и мир", "Анна Каренина"],
    "Иван Гончаров": ["Обломов"],
    "Александр Грибоедов": ["Горе от ума"],
    "Николай Карамзин": ["Бедная Лиза"],
    "Леонид Андреев": ["Рассказ о семи повешенных"],
    "Иван Бунин": ["Господин из Сан-Франциско"],
    "Евгений Замятин": ["Мы"],
    "Уильям Шекспир": ["Гамлет", "Ромео и Джульетта"],
    "Данте Алигьери": ["Божественная комедия"],
    "Иоганн Вольфганг Гёте": ["Фауст", "Страдания юного Вертера"],
    "Карл Маркс": ["Капитал"],
    "Фридрих Ницше": ["Так говорил Заратустра"],
    "Джейн Остин": ["Гордость и предубеждение"],
    "Марк Твен": ["Приключения Гекльберри Финна", "Приключения Тома Сойера"],
    "Оскар Уайльд": ["Портрет Дориана Грея"],
    "Артур Конан Дойл": ["Приключения Шерлока Холмса"],
    "Эдгар Аллан По": ["Убийство на улице Морг"],
    "Говард Лавкрафт": ["Зов Ктулху", "Тень над Иннсмутом", "Ужас в Данвиче"],
    "Джек Лондон": ["Мартин Иден"],
    "Франц Кафка": ["Процесс", "Превращение"],
    "Джордж Оруэлл": ["1984", "Скотный двор"],
    "Михаил Булгаков": ["Мастер и Маргарита", "Собачье сердце"]
}

def transliterate_author(full_name):
    mapping = {
        'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'е': 'e', 'ё': 'yo',
        'ж': 'zh', 'з': 'z', 'и': 'i', 'й': 'y', 'к': 'k', 'л': 'l', 'м': 'm',
        'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u',
        'ф': 'f', 'х': 'kh', 'ц': 'ts', 'ч': 'ch', 'ш': 'sh', 'щ': 'shch',
        'ъ': '', 'ы': 'y', 'ь': '', 'э': 'e', 'ю': 'yu', 'я': 'ya', ' ': '_'
    }
    parts = full_name.strip().split()
    if len(parts) < 2:
        raise ValueError(f"Имя должно содержать как минимум имя и фамилию: {full_name}")

    last_name = parts[-1].lower()
    first_name = parts[0].lower()

    last_trans = ''.join(mapping.get(c, '') for c in last_name)
    first_trans = ''.join(mapping.get(c, '') for c in first_name)

    return f"{last_trans}_{first_trans}"

def transliterate_title(title):
    mapping = {
        'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'е': 'e', 'ё': 'yo',
        'ж': 'zh', 'з': 'z', 'и': 'i', 'й': 'y', 'к': 'k', 'л': 'l', 'м': 'm',
        'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u',
        'ф': 'f', 'х': 'kh', 'ц': 'ts', 'ч': 'ch', 'ш': 'sh', 'щ': 'shch',
        'ъ': '', 'ы': 'y', 'ь': '', 'э': 'e', 'ю': 'yu', 'я': 'ya'
    }

    title = title.lower().strip()
    title = re.sub(r'[^\w\s-]', '', title)
    title = re.sub(r'[-\s]+', '_', title)

    result = ''.join(mapping.get(char, char) for char in title)
    return re.sub(r'_+', '_', result).strip('_')

def clean_text(text):
    start_phrase = "Спасибо, что скачали книгу в бесплатной электронной библиотеке Royallib.ru"
    end_phrase = "Приятного чтения!"

    start_idx = text.find(start_phrase)
    if start_idx != -1:
        end_idx = text.find(end_phrase, start_idx)
        if end_idx != -1:
            text = text[:start_idx] + text[end_idx + len(end_phrase):]

    return text.strip().lower()

session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
})

for author, titles in books_by_author.items():
    try:
        author_folder = transliterate_author(author)
    except Exception as e:
        print(f"❌ Не удалось обработать имя автора '{author}': {e}")
        continue

    for title in titles:
        title_clean = transliterate_title(title)

        book_page_url = f"https://royallib.com/book/{author_folder}/{title_clean}.html"
        download_url = f"https://royallib.com/get/txt/{author_folder}/{title_clean}.zip"

        print(f"\n🔍 Обработка: {title} by {author}")
        print(f"  📄 Страница книги: {book_page_url}")
        print(f"  💾 ZIP-архив: {download_url}")

        try:
            print("  → Посещение страницы книги...")
            response = session.get(book_page_url, timeout=15)
            if response.status_code != 200:
                print(f"  ❌ Ошибка доступа к странице книги: {response.status_code}")
                continue
            time.sleep(1.5)

            session.headers.update({'Referer': book_page_url})
            print("  → Скачивание ZIP...")
            response = session.get(download_url, stream=True, timeout=15)

            if response.status_code != 200:
                print(f"  ❌ Ошибка скачивания ZIP: {response.status_code}")
                continue

            zip_path = os.path.join(zip_dir, f"{title_clean}.zip")
            with open(zip_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"  ✅ Архив сохранён: {zip_path}")

            if not zipfile.is_zipfile(zip_path):
                print("  ❌ Файл не является ZIP-архивом.")
                with open(zip_path, 'r', encoding='utf-8', errors='ignore') as f:
                    head = f.read(500)
                    print(f"  🔍 Начало файла:\n{head}")
                continue

            with open(zip_path, 'rb') as f:
                raw_data = f.read(1024)

            if raw_data.startswith(b'PK'):
                print("  ✅ Это настоящий ZIP")
            else:
                try:
                    html_preview = raw_data.decode('cp1251', errors='ignore')
                    print(f"  ❌ Это НЕ ZIP, а HTML:\n{html_preview[:500]}")
                except:
                    print(f"  ❌ Не удалось распознать содержимое: {raw_data[:100]}")

            extracted_dir = os.path.join(zip_dir, f"{title_clean}_extracted")
            os.makedirs(extracted_dir, exist_ok=True)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extracted_dir)

            txt_files = [f for f in os.listdir(extracted_dir) if f.endswith('.txt')]
            if not txt_files:
                print("  ❌ TXT-файл не найден в архиве")
                continue

            txt_path = os.path.join(extracted_dir, txt_files[0])

            content = read_with_encoding(txt_path)

            print("🔍 Первые 500 символов ДО очистки:")
            print(repr(content[:500]))

            cleaned_content = clean_text(content)

            print("\n🔍 Первые 500 символов ПОСЛЕ очистки:")
            print(repr(cleaned_content[:500]))

            output_filename = f"{title} by {author}.txt"
            output_path = os.path.join(txt_dir, output_filename)
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(cleaned_content)

            print(f"  ✅ Сохранено: {output_path}")

            shutil.rmtree(extracted_dir)
            os.remove(zip_path)

        except Exception as e:
            print(f"  ⚠️ Ошибка при обработке '{title}': {e}")

print("\n🎉 Все книги обработаны!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

🔍 Обработка: Капитанская дочка by Александр Пушкин
  📄 Страница книги: https://royallib.com/book/pushkin_aleksandr/kapitanskaya_dochka.html
  💾 ZIP-архив: https://royallib.com/get/txt/pushkin_aleksandr/kapitanskaya_dochka.zip
  → Посещение страницы книги...
  → Скачивание ZIP...
  ✅ Архив сохранён: /content/drive/MyDrive/tf-idf/zip/kapitanskaya_dochka.zip
  ✅ Это настоящий ZIP
🔍 Первые 500 символов ДО очистки:
'Спасибо, что скачали книгу в бесплатной электронной библиотеке Royallib.ru: http://royallib.ru\n\nВсе книги автора: http://royallib.ru/author/pushkin_aleksandr.html\n\nЭта же книга в других форматах: http://royallib.ru/book/pushkin_aleksandr/kapitanskaya_dochka.html\n\nПриятного чтения!\n\n\n\nБереги честь смолоду.\n\n\xa0\xa0\xa0\xa0Пословица\n\n\n\nГлава I\n\nСержант гвардии[1 - Гвардия – специальные отборные войска. Первые гвардейские полки (Семено

KeyboardInterrupt: 

# To lowercase manual

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

folder_path = '/content/drive/MyDrive/tf-idf/txt/'

try:
    import chardet
except ImportError:
    print("Устанавливаем библиотеку chardet...")
    !pip install chardet
    import chardet

if not os.path.exists(folder_path):
    print(f"Папка не найдена: {folder_path}")
else:
    txt_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.txt')]

    if not txt_files:
        print("В папке нет .txt файлов.")
    else:
        print(f"Найдено файлов: {len(txt_files)}")

        for filename in txt_files:
            file_path = os.path.join(folder_path, filename)

            try:
                with open(file_path, 'rb') as file:
                    raw_data = file.read()

                detected = chardet.detect(raw_data)
                encoding = detected['encoding']
                confidence = detected['confidence']

                print(f"{filename}: определена кодировка {encoding} (уверенность: {confidence:.2f})")

                try:
                    text = raw_data.decode(encoding)
                except (UnicodeDecodeError, TypeError):
                    print(f"  Ошибка декодирования {filename} как {encoding}, пробуем utf-8...")
                    text = raw_data.decode('utf-8', errors='replace')

                lower_text = text.lower()

                with open(file_path, 'w', encoding='utf-8') as file:
                    file.write(lower_text)

                print(f"✅ Обработан и сохранён в UTF-8: {filename}")

            except Exception as e:
                print(f"❌ Ошибка при обработке {filename}: {e}")

print("Готово!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Найдено файлов: 26
Капитанская дочка by Александр Пушкин.txt: определена кодировка utf-8 (уверенность: 0.99)
✅ Обработан и сохранён в UTF-8: Капитанская дочка by Александр Пушкин.txt
Евгений Онегин by Александр Пушкин.txt: определена кодировка utf-8 (уверенность: 0.99)
✅ Обработан и сохранён в UTF-8: Евгений Онегин by Александр Пушкин.txt
Муму by Иван Тургенев.txt: определена кодировка utf-8 (уверенность: 0.99)
✅ Обработан и сохранён в UTF-8: Муму by Иван Тургенев.txt
Вий by Николай Гоголь.txt: определена кодировка utf-8 (уверенность: 0.99)
✅ Обработан и сохранён в UTF-8: Вий by Николай Гоголь.txt
Обломов by Иван Гончаров.txt: определена кодировка utf-8 (уверенность: 0.99)
✅ Обработан и сохранён в UTF-8: Обломов by Иван Гончаров.txt
Тарас Бульба by Николай Гоголь.txt: определена кодировка utf-8 (уверенность: 0.99)
✅ Обработан и сохранён в UTF-8: Тарас 